In [1]:
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import base64
import papermill as pm
import os 
import sys
import re 
import datetime
import pickle
import nbconvert
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter
from nbconvert.writers import FilesWriter
%load_ext autoreload
%autoreload 2 

## Run Notebooks

In [2]:
today_str = str(datetime.datetime.now().date())

In [3]:
current_path_split = os.getcwd().split('/')
if current_path_split[-1] == 'notebooks':
    os.chdir('..')
    print('Changed dir')
    print(os.getcwd())
elif current_path_split[-1] == 'run_notebooks':
    os.chdir('../..')
    print('Changed dir')
    print(os.getcwd())


Changed dir
/home/malcolm/meh_scraper


In [4]:
db_location = 'data/meh_scraper.db'

In [5]:
!pwd

/home/malcolm/meh_scraper


In [6]:
tm_notebook = "notebooks/Meh_Analysis_v1.ipynb"
tm_new_notebook = f"notebooks/run_notebooks/Meh_Analysis_v1_{today_str}.ipynb"

In [7]:
try:
    pm.execute_notebook(tm_notebook
              , tm_new_notebook
              , {"db_location": db_location})
    tm_error = None
except Exception as e:
    tm_error = str(e)
    print("Error")
    print(tm_error)

## Email

In [8]:
class EmailSender():
    
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
        
    @staticmethod
    def create_notebook_html(new_notebook_name):

        with open(new_notebook_name, 'r') as hnd:
            notebook_txt = hnd.read()
        nb_notebook = nbformat.reads(notebook_txt, as_version=4)
#         HTMLExporter.template_file = 'basic'
        html_exporter = HTMLExporter()
        html_exporter.template_file = 'basic'
        (body, resources) = html_exporter.from_notebook_node(nb_notebook)

        writer1 = FilesWriter()
        html_name = re.sub('.ipynb', '', new_notebook_name)
        writer1.write(output = body, resources = resources, notebook_name=html_name)
        html_name = html_name + '.html'

        return(html_name)

    @staticmethod
    def create_html_attachment(html_name):

        with open(html_name, 'r') as hnd:
            html = hnd.read()
        part1 = MIMEText(html, 'html')
        part1.add_header("Content-Disposition", f"attachment; filename= {html_name}")

        return(part1)
    
    def setup_email_params(self):
        """
        Set up message and server for sending email 
        """
        # Set up Email Structure 
        self.message = MIMEMultipart()
        self.message["Subject"] = self.Subject
        self.message["From"] = self.sender_email
        self.message["To"] = self.reciever_email
        # Email Body 
        text = MIMEText(self.Body, "plain")
        self.message.attach(text)
        # Run Notebook html 
        if 'run_notebook_path' in self.__dict__.keys():
            self.message.attach(self.html_attachment)
        # Set up Server 
        port = 465
        context = ssl.create_default_context()
        self.server = smtplib.SMTP_SSL("smtp.gmail.com", port, context=context)
        
        
    def send_email(self):
        
        self.server.login(self.gmail_login, self.pw)
        self.server.sendmail(self.gmail_login, self.reciever_email, self.message.as_string())
        
    def execute(self):
        
        if 'run_notebook_path' in self.__dict__.keys():
            html_path = self.create_notebook_html(self.run_notebook_path)
            self.html_attachment = self.create_html_attachment(html_path)
        
        self.setup_email_params()
        self.send_email()

In [9]:
 
message_params = {}
message_params['gmail_login'] = "malctaylordev@gmail.com"
message_params['sender_email'] = "malctaylordev@gmail.com"
message_params['reciever_email'] = "malctaylor15@gmail.com"
with open('/home/malcolm/credentials/email_pw.pkl', 'rb') as hnd:
    message_params['pw'] = pickle.load(hnd)['pw']
message_params['Subject'] = "Error in Meh Analysis: "+today_str
message_params['Body'] = """ 
        There was an error in Meh Analysis Notebook. 

        Stack Trace: 
""" + str(tm_error)
message_params['run_notebook_path'] = tm_new_notebook

In [10]:
email_sender = EmailSender(**message_params)
email_sender.execute()
